### As the basic for the experiment it is take 6 bit cipher with 2 parallel Sboxes and P layer [0, 3, 4, 1, 2, 5]

In [3]:
def permute_6bit(state: int) -> int:
    """
    Permutes a 6-bit integer according to the mapping [0, 3, 4, 1, 2, 5].
    Here, bit position 0 is the most-significant bit.
    """
    bits = [(state >> (5 - i)) & 1 for i in range(6)]
    mapping = [0, 3, 4, 1, 2, 5]
    out_bits = [0] * 6
    for i, target in enumerate(mapping):
        out_bits[target] = bits[i]
    result = 0
    for bit in out_bits:
        result = (result << 1) | bit
    return result

def sbox_layer(state: int, sbox_left: list, sbox_right: list) -> int:
    """
    Applies S-box substitution on the two 3-bit halves of the state.
    The left half consists of the top 3 bits, and the right half consists of the bottom 3 bits.
    sbox_left and sbox_right are lists of 8 integers (each representing a 3-bit output).
    """
    left = (state >> 3) & 0b111
    right = state & 0b111
    left_sub = sbox_left[left]
    right_sub = sbox_right[right]
    return (left_sub << 3) | right_sub

def toy_cipher_encrypt(plaintext: int, round_keys: list, sboxes, apply_whitening: bool = True) -> int:
    """
    Encrypts a 6-bit plaintext using a toy cipher with multiple rounds.

    Each round performs:
      1. Key addition: XOR the state with the round key. The first key is treated as a whitening key.
         When apply_whitening is False, the key addition for the first round is skipped.
      2. S-box substitution: The state is split into two 3-bit halves, each substituted using the provided S-box.
      3. Permutation: The 6-bit state is permuted using the mapping [0, 3, 4, 1, 2, 5].

    Parameters:
      plaintext: A 6-bit integer (0 <= plaintext < 64) representing the input.
      round_keys: A list of 6-bit integers (each between 0 and 63) to be used as keys for each round.
                  The first key is the whitening key.
      sboxes: Either a single S-box (a list of 8 integers) used for both halves in every round,
              or a list of pairs (sbox_left, sbox_right) for each round.
      apply_whitening: If True, the first (whitening) key is applied; if False, it is skipped.
    
    Returns:
      The ciphertext as a 6-bit integer.
    """
    state = plaintext
    num_rounds = len(round_keys)
    
    # Determine if a single S-box is provided or a pair for each round.
    if isinstance(sboxes[0], int):
        sboxes_per_round = [(sboxes, sboxes)] * num_rounds
    else:
        sboxes_per_round = sboxes
    
    for round_index in range(num_rounds):
        # Apply key addition unless it's the first round and whitening is disabled.
        if round_index != 0 or apply_whitening:
            state ^= round_keys[round_index]
        
        # S-box layer.
        sbox_left, sbox_right = sboxes_per_round[round_index]
        state = sbox_layer(state, sbox_left, sbox_right)
        
        # Permutation layer.
        state = permute_6bit(state)
    
    return state

# Example usage:
if __name__ == "__main__":
    # Sample S-box: must be a permutation of 0 through 7.
    sample_sbox = [0, 6, 4, 1, 7, 2, 5, 3]
    
    # Example: Using 2 rounds with these 6-bit round keys.
    # The first key is considered the whitening key.
    round_keys = [0b101011, 0b010101]
    
    # Encrypt a 6-bit plaintext (for example, 0b110010).
    plaintext = 0b110010

    # Encrypt with whitening applied (default).
    ciphertext_whitened = toy_cipher_encrypt(plaintext, round_keys, sample_sbox, apply_whitening=True)
    
    # Encrypt with whitening skipped.
    ciphertext_no_whitening = toy_cipher_encrypt(plaintext, round_keys, sample_sbox, apply_whitening=False)
    
    print("Plaintext:           {:06b}".format(plaintext))
    print("Ciphertext (with whitening):    {:06b}".format(ciphertext_whitened))
    print("Ciphertext (no whitening):      {:06b}".format(ciphertext_no_whitening))


Plaintext:           110010
Ciphertext (with whitening):    101101
Ciphertext (no whitening):      101111


In [ ]:

def truth_table_test(whithening = False, rounds = 2):
    f = []
    key = [random.randint(1, 10) for _ in range(rounds)]
    cipher = SmallPresent(key, 2, rounds)
    start_time = time.time()
    y_bit = 0
    for input_key in range(2**(8*(rounds))):
        for input_state in range(2**8-1,-1,-1):
        #print(key, state)
            y = cipher.encrypt_integer_key(input_state, input_key, True) 
            y_bit = (y_bit<<1) | ((y) & 1)
        f.append(y_bit)
        y_bit = 0
        
    

    return f